In [1]:
import cv2
import matplotlib.pyplot as plt
from copy import copy
import pandas as pd
import os
from face_detection import detect_faces, compute_face_areas_with_percents, cascade

# В чем вообще идея:

### Если я пройдусь по всем файлам из директории frames_2012, я смогу создать датафрейм с колонками "название файла", "количество лиц", "процент, который занимают лица на картинке". В дальнейшем можно было бы построить гистограмму распределения, посмотреть, что превалирует (распределение количества лиц, распредение процентажа) + построить график того, в каких моментах фильма процентаж от одного лица наиболее высокий -> так я могла бы находить на протяжении фильма крупные планы (например). В целом это могло бы быть актуально в случае попытки находить крупные планы не только людей, но и вещей, в таком случае мы видели бы, как толстовская детальность передается в фильме.

### Кроме того, я могла бы анализировать цветовую передачу кадров - там, где сумма значений пикселей на напротяжении нескольких кадров равна нулю, скорее всего есть склейка монтажная. Тогда можно было бы проверить, насколько быстрое повествование, у кого кадры (сцены) длиннее, у кого наоборот короче.

##### Следующие чанки с функциями и классификатором - временные, они будут перенесены в отдельный файл .py, откуда я буду их импортировать (хотя там возникает проблема с функцией classificator, файлы из которой по сути потом использует функция detect_faces

In [39]:
def classificator(name):
    cascade = cv2.CascadeClassifier(name)
    return cascade

def detect_faces(cascade, image, scaleFactor = 1.1, return_faces=False): 
    '''
    :scaleFactor: то, во сколько раз мы уменьшаем наше окно поиска лиц или чего-либо еще при каждой итерации
    '''
    
    '''на всякий случай поработаем с копией'''
    image_copy = image.copy()

    '''копию картинки переводим в серый, потому что detectMultiScale
       берет на фход только серые изображения'''
    gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)

    '''Детектим лица с помощью haar classifier''' 
    faces = cascade.detectMultiScale(gray_image, scaleFactor=scaleFactor, minNeighbors=5)
    
#     print("Лиц обнаружено: " + format(len(faces)))

    for (x, y, w, h) in faces:
        cv2.rectangle(image_copy, (x, y), (x+w, y+h), (0, 255, 0), 3)

    return (image_copy, faces) if return_faces else image_copy

def compute_face_areas_with_percents(array_of_faces, image):
    '''This function computes rectangle area and the percentage of it.
    It does not work without function detect_faces, which provides the array of faces (coordinates)'''
    
    img_h, img_w = image.shape[:2] #we do not need rgb size
    img_area = img_h * img_w
    areas = []
    percentages_of_area = []
    for face in array_of_faces:
        w, h = face[2], face[3]
        area = w * h
        percent = area / img_area * 100
        percentages_of_area.append(percent)
        areas.append(area)
    return areas, percentages_of_area

In [45]:
frontal_face_cascade = classificator('../haarcascade_frontalface_default.xml')
profile_face_cascade = classificator('../haarcascade_profileface.xml')

## Создадим функцию, которая раскладывает наше видео на кадры

In [4]:
def frame_extraction(film_path):
    vidcap = cv2.VideoCapture(film_path)
    success,image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite("frames_2012/frame_{}.jpg".format(count), image)
        success,image = vidcap.read()
#         print('Read a new frame: ', success, ' ', count) #это вероятно можно убрать
        count += 1

## Попробуем разложить трейлер "Анны Карениной" на кадры, которые сохранятся в папке frames_2012

In [5]:
film_path = 'Anna_Karenina_Official_Trailer.mp4'

In [6]:
'''на выдаче пока не поняла, почему последнее число 3615, если всего кадров должно быть 3624 (ведь длина трейлера - 2 мин 31 сек)
UPD: gjx'''
frame_extraction(film_path)

## Теперь к каждому кадру применим наши функции из скрипта Karenina_OpenCV_practice (которые пока что записаны выше). Результат запишем в датафрейм.

In [48]:
columns_name = ['frame_num', 'num_of_faces', 'percentage']
data = pd.DataFrame(columns = columns_name)

In [58]:
directory = 'frames_2012'
for file in os.listdir(directory):
    if file.endswith('.jpg'):
        image = cv2.imread(os.path.join(directory, file)) #читаем файл, получаем матрицу
        image, faces = detect_faces(frontal_face_cascade, image, 1.2, True) #находим лица на картинке, 
                                                                            #получаем координаты картинки и координаты лиц
        result = compute_face_areas_with_percents(faces, image) #подсчитываем площадь (пиксели в кв) и процент от картинки
        
        dict_of_frames = {'frame_num': file, 'num_of_faces': len(result[0]), 'percentage': result[1]}
#         print(dict_of_frames)
        data = data.append(dict_of_frames, ignore_index=True)

In [66]:
'''Возникшие проблемы: 
1) надо понять, как это нормально отсортировать, вероятно, упорядочивать надо еще на этапе обхода директории (странно, что он рандомно
   файлы располагает, а не по порядку, как они лежат у меня в папочке)
2) Надо улучшить df, потому что пока что наличие списка в значении процентажа не совсем правильно и не очень красиво
3) проблема с детектингом - все-таки эта программа совершает значительные ошибки в распознавании лиц
4) надо обрезать черные рамки сверху и снизу кадра
5) надо придумать, что делать с черными кадрами, потому что если их несколько, то скорее всего это склейка, и тогда эту информацию 
можно как-то использовать, но пока что эти кадры просто занимаю место в df'''

data

,frame_num,num_of_faces,percentage
0,frame_3284.jpg,0,[]
1,frame_1493.jpg,0,[]
2,frame_2824.jpg,5,"[0.5469835069444444, 0.6433376736111112, 0.304..."
3,frame_2830.jpg,0,[]
4,frame_1487.jpg,0,[]
...,...,...,...
3611,frame_3289.jpg,0,[]
3612,frame_967.jpg,1,[0.1181640625]
3613,frame_2829.jpg,5,"[0.5625, 0.3282335069444445, 0.6103515625, 0.7..."
3614,frame_973.jpg,1,[0.1181640625]
